###CROP RECOMMENDATION SYSTEM###

In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/soil-image-classification/test/Alluvial soil/es-alluvial-soil-01c.jpg
/kaggle/input/soil-image-classification/test/Alluvial soil/Alluvial-Soil.jpg
/kaggle/input/soil-image-classification/test/Alluvial soil/An-example-of-a-soil-profile-from-a-well-drained-alluvial-fan-in-the-Atacama-Desert.png
/kaggle/input/soil-image-classification/test/Alluvial soil/AlluvialsoilsRicker4Alluvium_LandUse1200x630.jpg
/kaggle/input/soil-image-classification/test/Alluvial soil/38.jpg
/kaggle/input/soil-image-classification/test/Alluvial soil/th4.jpg
/kaggle/input/soil-image-classification/test/Alluvial soil/OIP.95L3kBHrnJiscNzRwQgmlgHaDW.jpg
/kaggle/input/soil-image-classification/test/Alluvial soil/Alluvial-Soil..png
/kaggle/input/soil-image-classification/test/Alluvial soil/alluvial-deposits-david-parkerscience-photo-library.jpg
/kaggle/input/soil-image-classification/test/Alluvial soil/th9.jpg
/kaggle/input/soil-image-classification/test/Alluvial soil/th34.jpg
/kaggle/input/soil-image-clas

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam

# Path to dataset
train_dir = '/kaggle/input/soil-image-classification/train_data'
test_dir = '/kaggle/input/soil-image-classification/test'

# Image Data Generator
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Load MobileNetV2 base
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base layers

# Define classifier
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(train_data.num_classes, activation='softmax')
])

# Compile model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_data, validation_data=test_data, epochs=10)

# Save the model
model.save("soil_classifier_model.h5")


Found 343 images belonging to 4 classes.
Found 158 images belonging to 4 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.3545 - loss: 1.5147 - val_accuracy: 0.7152 - val_loss: 0.6573
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.7904 - loss: 0.5795 - val_accuracy: 0.8228 - val_loss: 0.4830
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.8707 - loss: 0.3958 - val_accuracy: 0.8671 - val_loss: 0.3841
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.9130 - loss: 0.2758 - val_accuracy: 0.8797 - val_loss: 0.3439
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9404 - loss: 0.2026 - val_accuracy: 0.8797 - val_loss: 0.3233
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.9852 - loss: 0.1570 - val_accuracy: 0.8734 - val_loss: 0.3310
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.9793 - loss: 0.1287 - val_accuracy: 0.8924 - val_loss: 0.2870
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 996ms/step - accuracy: 0.9827 - loss: 0.0988 - val_accuracy: 0.8987 - val_loss: 0.3157


In [6]:
print(train_data.class_indices)


{'Alluvial Soil': 0, 'Black Soil': 1, 'Clay Soil': 2, 'Red Soil': 3}


In [21]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load model
model = load_model("soil_classifier_model.h5")

# Class map
soil_classes = {
    0: "Alluvial Soil",
    1: "Black Soil",
    2: "Clay Soil",
    3: "Red Soil"
}

def predict_soil(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    return soil_classes[class_idx]

# Example
print("Detected Soil Type:", predict_soil("/kaggle/input/example-jpg/image_2025-04-08_201349630.png"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Detected Soil Type: Alluvial Soil


In [10]:
import pandas as pd

df=pd.read_csv("/kaggle/input/crop-recommendation/Crop_recommendation.csv")
df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X = df.drop('label', axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print("Model Accuracy:", accuracy)


Model Accuracy: 0.9931818181818182


In [12]:
def recommend_crop(N, P, K, temp, humidity, ph, rainfall):
    data = [[N, P, K, temp, humidity, ph, rainfall]]
    return model.predict(data)[0]


In [13]:
# Try recommending
recommend_crop(90, 42, 43, 20.5, 80, 6.5, 150)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


'jute'

###WEATHER ADVISORY###

In [14]:
import requests
from kaggle_secrets import UserSecretsClient

# Get your API key from Kaggle secrets
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("weather_api")

def get_weather(city):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url).json()
    
    # Extract relevant data
    temp = response["main"]["temp"]
    humidity = response["main"]["humidity"]
    condition = response["weather"][0]["main"]

    # Simple weather-based farming advice
    if temp > 35:
        advice = "🔥 It's too hot. Consider frequent irrigation and mulching."
    elif temp < 10:
        advice = "❄️ Low temperature. Protect sensitive crops with covers."
    elif humidity > 80:
        advice = "🌧️ High humidity! Risk of fungal diseases – consider fungicide spraying."
    else:
        advice = "✅ Optimal conditions for most crops."

    # Return all the juicy info
    return {
        "Temperature (°C)": temp,
        "Humidity (%)": humidity,
        "Condition": condition,
        "Advice": advice
    }


In [15]:
weather_report = get_weather("Nainital")  # or try "Delhi", "Chennai", etc.
print(weather_report)


{'Temperature (°C)': 14.16, 'Humidity (%)': 26, 'Condition': 'Clear', 'Advice': '✅ Optimal conditions for most crops.'}


In [16]:
for key, value in weather_report.items():
    print(f"{key}: {value}")


Temperature (°C): 14.16
Humidity (%): 26
Condition: Clear
Advice: ✅ Optimal conditions for most crops.


In [24]:
def recommend_crop_with_weather(city):
    # Example weather data — replace with real API if you have one
    temperature = 28.0
    humidity = 70.0
    rainfall = 200.0
    ph = 6.5
    nitrogen = 80
    potassium = 40
    phosphorous = 60

    # 🌟 Make sure this is a 2D array: shape (1, 7) for a single prediction
    input_data = np.array([[nitrogen, phosphorous, potassium, temperature, humidity, ph, rainfall]])

    # Ensure it's float32 for TensorFlow
    input_data = input_data.astype(np.float32)

    prediction = model.predict(input_data)[0]
    
    return {
        "Best Crop (Weather-Based)": prediction
    }


In [25]:
result = recommend_crop_with_weather("Pune")
for key, value in result.items():
    print(f"{key}: {value}")

ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None

In [26]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load trained soil classifier
soil_model = load_model("soil_classifier_model.h5")

# Soil-crop map
soil_crop_map = {
    "Alluvial": "Wheat",
    "Black": "Cotton",
    "Red": "Millets",
    "Sandy": "Groundnut",
    "Laterite": "Cashew",
    "Peaty": "Rice"
}

# Class index map
soil_classes = {0: "Alluvial", 1: "Black", 2: "Red", 3: "Sandy"}  # Update if more classes

def predict_soil_type(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = soil_model.predict(img_array)
    class_idx = np.argmax(pred)
    return soil_classes[class_idx]

def recommend_crop_final(image_path, location):
    soil_type = predict_soil_type(image_path)
    crop_from_soil = soil_crop_map.get(soil_type, "Maize")

    print(f"🧪 Detected Soil Type: {soil_type}")
    print(f"🌱 Recommended Crop (Soil-Based): {crop_from_soil}")

    # Weather integration
    result = recommend_crop_with_weather(location)
    for key, value in result.items():
        print(f"{key}: {value}")
    
    print(f"🎯 Final Suggestion: Base on both soil & weather -> {crop_from_soil}")

# Example usage
recommend_crop_final("/kaggle/input/example-jpg/image_2025-04-08_201349630.png", "Pune")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
🧪 Detected Soil Type: Alluvial
🌱 Recommended Crop (Soil-Based): Wheat


ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None